In [1]:
import numpy as np
import csv
import math
import cv2
import pickle
from collections import defaultdict
from datetime import datetime, timedelta
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.segmentation import find_boundaries
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from numpy.linalg import inv
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier

In [2]:
def file_reading(file_path):
    #reading annotations file
    ann_file = open(file_path,"r") #opening file in read mode only
    strings = [x.strip() for x in ann_file.readlines()]
    stimes=[]
    etimes=[]
    for i in range(len(strings)):
        s1,s2=strings[i].split("-")
        stimes.append(s1.strip())
        etimes.append(s2.strip())
    return stimes,etimes

In [3]:
def compare(stime,etime,abnormal_stimes,abnormal_etimes):
    length = len(abnormal_stimes)
    for i in range(length):
        t1 = datetime.strptime(abnormal_stimes[i], '%M:%S').time()
        t2 = datetime.strptime(abnormal_etimes[i], '%M:%S').time()
        obj1 = timedelta(hours=t1.hour, minutes=t1.minute, seconds=t1.second)
        obj2 = timedelta(hours=t2.hour, minutes=t2.minute, seconds=t2.second)
        if (stime >= obj1 and etime <= obj2) or (stime < obj1 and etime > obj1) or (stime < obj2 and etime > obj2):
            return 1
    return -1

In [4]:
class labeling_objects:
    def __init__(self,clip_no,stime,etime,label,interest_points,segments):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.label = label
        self.interest_points = interest_points
        self.segments = segments

In [5]:
def active_superpixels(flow,segments,frame):
    no_of_superpixels = segments.max()+1
    #print(no_of_superpixels)
    height,width = segments.shape
    count = [0]*no_of_superpixels
    points = []
    threshold = [5.00000000e-05,5.00000000e-05]
    #threshold = 2.00000000e-05
    #mag = cv2.cartToPolar(flow[...,0],flow[...,1])[0]
    
    for i in range(height):
        for j in range(width):
            if abs(flow[i][j][0]) > threshold[0] or abs(flow[i][j][1]) > threshold[0]:
            #if mag[i][j]>threshold:
                points.append([i,j])
    #new_frame = frame
    points = np.asarray(points)
    #print("points array shape",points.shape)
    
    for index in range(len(points)):
        seg = segments[points[index][0]][points[index][1]]
        count[seg]+= 1
        #new_frame = cv2.circle(new_frame,tuple(points[index]),2,(0,0,255), -1)

    active_superpixels = [True]*no_of_superpixels
    for i in range(no_of_superpixels):
        total_count = np.count_nonzero(segments==i) #counting total no of pixels in superpixel i
        #print("total no of pixels in superpixel",i,"is ",total_count)
        #print("active pixels in superpixel",i,"is ",count[i])
        if count[i] >= 0.4*total_count:
            active_superpixels[i] = False

    #print("Total no of superpixels ",no_of_superpixels)
    #cv2.imshow('frame',new_frame)
    #cv2.waitKey(3000)
    #cv2.destroyAllWindows()
    return active_superpixels

In [6]:
video_path = 'E:\\Study\\Sem Project\\Data\\traffic-junction.avi'
ann_file_path = "E:\\Study\\Sem Project\\Data\\abnormal_times.txt"
cap = cv2.VideoCapture(video_path)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 500,   # How many pts. to locate
                       qualityLevel = 0.1,  # b/w 0 & 1, min. quality below which everyone is rejected
                       minDistance = 7,   # Min eucledian distance b/w corners detected
                       blockSize = 3 ) # Size of an average block for computing a derivative covariation matrix over each pixel neighborhood

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),  # size of the search window at each pyramid level
                  maxLevel = 2,   #  0, pyramids are not used (single level), if set to 1, two levels are used, and so on
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

''' Criteria : Termination criteria for iterative search algorithm.
    after maxcount { Criteria_Count } : no. of max iterations.
    or after { Criteria Epsilon } : search window moves by less than this epsilon '''

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

clip = 0
count = 0  # for the frame count
n = 50  # Frames refresh rate for feature generation

interest_points = []
history_all_clips = [] # to track the history of frames.

#for labeling the clips 
abnormal_stimes,abnormal_etimes = file_reading(ann_file_path)
secs=0
label_objects_array = []
frames = []
active_super_pixels_array = []
seg_array = []

while True:
    ret,frame = cap.read()
    if frame is None:
        break
    frames.append(frame)
    count+=1
    
    #cutting the clip, finding the interest points for last frame
    if count%n == 0 or (len(frames)!=0 and (count == total_frames)):
        if(len(frames)==50):
            secs = secs+2
            stime = timedelta(seconds = secs-2)
            etime = timedelta(seconds = secs)
            label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
        else:
            secs = secs+1
            stime = timedelta(seconds = secs-1)
            etime = timedelta(seconds = secs)
            label = compare(stime,etime,abnormal_stimes,abnormal_etimes)
        
        len_frame = len(frames)
        old_frame = frames[len_frame-1]
        # Convert to Grey Frame
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        segments = slic(old_frame, n_segments = 144, sigma = 5,compactness = 25)
        seg_array.append(segments)
        mask = np.zeros_like(old_frame)
        
        #features for ending frame
        p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        t = np.ones(shape = (len(p0),1))
        
        label_objects_array.append(labeling_objects(clip,stime,etime,label,p0[t==1],segments.copy()))
        clip += 1
        
        history_clip = defaultdict(list)
        #if count%10 == 0: # Refresh the tracking features after every 10 frames
        #    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        
        for i in range(len(frames)-1,0,-1):
            # calculate optical flow
            frame_new = frames[i]
            frame_gray = cv2.cvtColor(frame_new,cv2.COLOR_BGR2GRAY)
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
            if i==len(frames)-1:
                flow = cv2.calcOpticalFlowFarneback(old_gray,frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                flow=np.asarray(flow)
                active_super_pixels_array.append(active_superpixels(flow,segments,old_frame))
            # Select good points
            good_new = p1[st==1]
            good_old = p0[st==1]

            # draw the tracks
            for j,(new,old) in enumerate(zip(good_new,good_old)):
                if i == len(frames)-1:
                    history_clip[j].append(old)
                    history_clip[j].append(new)
                else:
                    history_clip[j].append(new)
                #a,b = new.ravel() #tmp new value
                #c,d = old.ravel() #tmp old value
                
                #draws a line connecting the old point with the new point
                #mask = cv2.line(mask, (a,b),(c,d), (0,0,255), 1)

                #draws the new point
                #frame = cv2.circle(frame,(a,b),2,(0,255,0), -1)
            """
            img = cv2.add(frame,mask)
            
            cv2.imshow('frame',img)
            k = cv2.waitKey(30) & 0xff

            #Show the Output
            if k == 27:
                cv2.imshow('', img)
                break
            """
            # Now update the previous frame and previous points
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1,1,2)
        frames = []
        history_all_clips.append(history_clip)

print("total frames ",count)
# release and destroy all windows
cv2.destroyAllWindows()
cap.release()

total frames  7824


In [7]:
print("total clips",clip)
"""
print("history length",len(history_all_clips))
print("label objects length",len(label_objects_array))
print("length of active super pixels array",len(active_super_pixels_array))
for i in range(len(active_super_pixels_array)):
    print("no of active superpixels in clip ",i,"is",np.count_nonzero(active_super_pixels_array[i]))
    """

total clips 157


'\nprint("history length",len(history_all_clips))\nprint("label objects length",len(label_objects_array))\nprint("length of active super pixels array",len(active_super_pixels_array))\nfor i in range(len(active_super_pixels_array)):\n    print("no of active superpixels in clip ",i,"is",np.count_nonzero(active_super_pixels_array[i]))\n    '

In [8]:
def region_formation(image):
    shape = image.shape
    x_val = shape[0]//3
    y_val = shape[1]//3
    regions_arr = image.reshape(int(shape[0]/x_val), x_val, -1, y_val).swapaxes(1,2).reshape(-1, x_val, y_val)
    return regions_arr

In [9]:
def small_length(val,arr):
    small_index = 0
    small_count = math.inf
    for i in range(len(arr)):
        if val in arr[i]:
            if small_count > len(arr[i]):
                small_count = len(arr[i])
                small_index = i
    return small_index

In [10]:
def in_list(c, region_list):
    for i in range(len(region_list)):
        if c in region_list[i]:
            return i
    return -1

In [11]:
def region_and_superpixels(segments):
    #dividing the entire frame into regions assigning each pixel location to region number
    #output: regions_ar- contains the information of superpixels belongs to that region
    #        region_segments - for a particular pixel location its associated region number.
    regions = region_formation(segments)
    region_index = defaultdict(set)
    x,m,n = regions.shape
    for i in range(x):
        for j in range(m):
            for k in range(n):
                region_index[i].add(regions[i][j][k])
    regions_ar = defaultdict(list)
    for i in range(len(region_index)):
        regions_ar[i]=list(region_index[i])
    for i in range(len(regions_ar)):
        for j in range(len(regions_ar[i])):
            region_num = small_length(regions_ar[i][j],regions)
            for ind in range(len(regions_ar)):
                if ind!=region_num and ind!=i:
                    if regions_ar[i][j] in regions_ar[ind]:
                        regions_ar[ind].remove(regions_ar[i][j])
    # printing here region number and its particular superpixels. 
    #for i in range(len(regions_ar)):
    #    print("region",i," ",regions_ar[i])
    region_segments = np.zeros_like(segments)
    for i in range(len(segments)):
        for j in range(len(segments[i])):
            region_segments[i][j] = in_list(segments[i][j],regions_ar)
    #for i in range(len(region_segments)):
    #    print(region_segments[i])
    return regions_ar,region_segments

In [12]:
region_segments_array = []
for i in range(len(label_objects_array)):
    regions_ar,region_segments = region_and_superpixels(label_objects_array[i].segments)
    region_segments_array.append(region_segments)
    #print()

In [18]:
codebook_all_clips = []
for i in range(len(history_all_clips)):
    #print(i,"th clip no of interest points",len(history_all_clips[i]))
    codebook_region = defaultdict(list)
    for j in range(len(history_all_clips[i])):
        history = history_all_clips[i][j]
        flow_value =[]
        #superpixel value 
        suppix = label_objects_array[i].segments[int(history[0][1])][int(history[0][0])]
        region = region_segments_array[i][int(history[0][1])][int(history[0][0])]
        for k in range(len(history)-1):
            flow_value.append([history[k][0]-history[k+1][0],history[k][1]-history[k+1][1]])
        flow_value = np.asarray(flow_value)
        mag = cv2.cartToPolar(flow_value[:,0],flow_value[:,1])[0]
        mag = mag.flatten()
        codebook_region[region].append(np.asarray(mag))
    region_array = np.asarray(list(codebook_region.values()))
    #print(len(region_array[0][0]))
    codebook_all_clips.append(region_array)

In [31]:
class GMM:
    
    def __init__(self,X,k,weights,means,variances,n_iter):
        self.X=X
        self.k=k
        self.weights=weights
        self.means = means
        self.variances = variances
        self.eps=1e-8
        self.n_iter = n_iter
        
    def run(self):
        for step in range(self.n_iter):
        
            likelihood=[]
            for j in range(self.k):
                likelihood.append(self.pdf(self.X, self.means[j], np.sqrt(self.variances[j])))
            likelihood = np.array(likelihood)
            
            b = []
            # Maximization step 
            
            for j in range(self.k):
                # use the current values for the parameters to evaluate the posterior
                # probabilities of the data to have been generanted by each gaussian    
                b.append((likelihood[j] * self.weights[j]) / (np.sum([likelihood[i] * self.weights[i] for i in range(self.k)],axis=0))+self.eps)

                # updage mean and variance
                self.means[j] = np.sum(b[j] * self.X) / (np.sum(b[j]+self.eps))
                self.variances[j] = np.sum(b[j] * np.square(self.X - self.means[j])) / (np.sum(b[j]+self.eps))
                
                #print("b dist=",b[j])
                # update the weights
                self.weights[j] = np.mean(b[j])
            #print(self.weights)
            return self

    def pdf(self,data,mean:float,variance:float):
        s1 = 1/(np.sqrt(2*np.pi*variance))
        print(len(data),mean)
        s2 = np.exp(-(np.square(data - mean)/(2*variance)))
        return s1*s2

In [32]:
#intial parameters for em clustering 
def intial_parameters(arr):
    k=4
    arr = np.asarray(arr)
    intial_weights = [(1-0.1)/k for i in range(k)]
    intial_weights = np.append(intial_weights,0.1) # adding extra weight element for background purpose
    means = np.random.choice(arr.flatten(),(k,2))
    means =np.append(means,[0,0]) # adding extra mean element for background purpose

    cov = np.random.sample(size=k)
    cov = np.append(cov,4.0) # adding extra cov element for background purpose
    return k,intial_weights,means,cov

In [33]:
class data_objects:
    def __init__(self,clip_no,stime,etime,weights_data,label):
        self.clip_no = clip_no
        self.stime = stime
        self.etime = etime
        self.weights_data = weights_data
        self.label = label

In [34]:
data_object_array=[]
loa = label_objects_array
for i in range(len(codebook_all_clips)):
    print("clip no ",i)
    clip_weights = []
    for j in range(len(codebook_all_clips[i])):
        k,initial_weights,means,cov = intial_parameters(codebook_all_clips[i][j])
        gmm = GMM(codebook_all_clips[i][j],k+1,initial_weights.copy(),means.copy(),cov.copy(),50)
        gmm.run()
        #print("wei",gmm.weights)
        if math.isnan(np.sum(gmm.weights)) == False:
            clip_weights.extend(gmm.weights)
    data_object_array.append(data_objects(loa[i].clip_no,loa[i].stime,loa[i].etime,np.asarray(clip_weights).flatten(),loa[i].label)) 
    print("weights",len(clip_weights))

clip no  0
31 0.0071419826708734035
31 0.06530773639678955
31 0.09849373251199722
31 0.022099848836660385
31 0.026878830045461655
47 0.03528357297182083
47 0.021478861570358276
47 0.01996714062988758
47 0.05967510864138603
47 0.03113095462322235
27 0.04283272847533226
27 0.050012312829494476
27 0.06453725695610046
27 0.03648988530039787
27 0.01037779450416565
8 0.02518828585743904
8 0.05949873849749565
8 0.0008971597417257726
8 0.15447898209095
8 0.44148364663124084
14 0.034614380449056625
14 0.006378726102411747
14 0.03413696214556694
14 0.06321913748979568
14 0.04582921788096428
7 0.046397048979997635
7 0.09675436466932297
7 0.0003021086158696562
7 0.01163441315293312
7 0.00883362628519535
5 5.92800235748291
5 5.052144527435303
5 5.208140850067139
5 9.039728164672852
5 4.802638530731201
7 9.049571990966797
7 0.006163720041513443
7 0.04720067232847214
7 6.639244556427002
7 0.018177777528762817
1 0.05495311692357063
1 0.01780293509364128
1 0.0658985823392868
1 0.04817020520567894
1 0.0

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


5 0.01201521884649992
5 0.4670464098453522
5 0.007440908811986446
5 1.3424261808395386
5 0.004661204293370247
1 0.018475867807865143
1 0.01602173037827015
1 0.010084779001772404
1 0.006497256923466921
1 0.003036192152649164
2 0.008284665644168854
2 0.03289569914340973
2 0.00021847261814400554
2 0.05298808589577675
2 0.008840160444378853
weights 45
clip no  7
33 0.000762939453125
33 0.0009237555204890668
33 0.007628249935805798
33 0.0031813043169677258
33 0.0022813070099800825
51 0.010021046735346317
51 0.001513872528448701
51 0.012634159997105598
51 0.009344064630568027
51 0.009458249434828758
30 0.0014921638648957014
30 0.0012368093011900783
30 0.0016991511220112443
30 0.0008884253329597414
30 0.001228362089022994
10 0.006964061874896288
10 0.5296547412872314
10 0.0018853095825761557
10 0.8939059376716614
10 0.35679423809051514
17 0.0032607808243483305
17 0.004426431376487017
17 0.0026083604898303747
17 0.001766270725056529
17 0.00037362618604674935
8 0.01645588129758835
8 0.015860870

ValueError: operands could not be broadcast together with shapes (45,) (49,) 

In [ ]:
total_data_objects = data_object_array
"""
print("total no of objects",len(total_data_objects))
for i in range(len(total_data_objects)):
    print("clip no=",total_data_objects[i].clip_no)
    print("start time=",total_data_objects[i].stime)
    print("end time=",total_data_objects[i].etime)
    print("weights =",total_data_objects[i].weights_data)
    print("label = ",total_data_objects[i].label,"\n")
    """

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
# normalizing weights.
for i in range(len(total_data_objects)):
    sum = np.sum(total_data_objects[i].weights_data)
    for j in range(len(total_data_objects[i].weights_data)):
        total_data_objects[i].weights_data[j] = total_data_objects[i].weights_data[j] / sum
    total_data_objects[i].weights_data = np.asarray(total_data_objects[i].weights_data)
    #print("sum ",i,"",np.sum(total_data_objects[i].weights_data))

In [ ]:
max_len = len(total_data_objects[0].weights_data)
for i in range(len(total_data_objects)):
    if max_len < len(total_data_objects[i].weights_data):
        max_len = len(total_data_objects[i].weights_data)
print("max len",max_len)

"""
min_len = len(total_data_objects[0].weights_data)
for i in range(len(total_data_objects)):
    if min_len > len(total_data_objects[i].weights_data):
        min_len = len(total_data_objects[i].weights_data)
print("min len",min_len)
"""

for i in range(len(total_data_objects)):
    for j in range(max_len - len(total_data_objects[i].weights_data)):
        total_data_objects[i].weights_data = np.append(total_data_objects[i].weights_data,0)

for i in range(len(total_data_objects)):
    #print("clip no",i)
    for j in range(len(total_data_objects[i].weights_data)):
        if total_data_objects[i].weights_data[j] == 0:
            total_data_objects[i].weights_data[j] = np.mean([total_data_objects[ind].weights_data[j] for ind in range(len(total_data_objects))])
"""
for i in range(len(total_data_objects)):
    for j in range(len(total_data_objects[i].weights_data)-min_len):
        total_data_objects[i].weights_data = np.delete(total_data_objects[i].weights_data,j)

print("after preprocessing lengths")
for i in range(len(total_data_objects)):
    print(len(total_data_objects[i].weights_data))
"""

In [ ]:
#dividing the total dataset into 60% training and 40% test
total_no = len(total_data_objects)
print("total objects=",total_no)
train_no=round(0.6*total_no)
print("train objects=",train_no)
test_no=round(0.4*total_no)
print("test objects=",test_no)

X_train=[]
X_test=[]
Y_train=[]
Y_test=[]

np.random.shuffle(total_data_objects)
for i in range(train_no):
    X_train.append(np.asarray(total_data_objects[i].weights_data))
    Y_train.append(total_data_objects[i].label)
for i in range(train_no,total_no):
    X_test.append(total_data_objects[i].weights_data)
    Y_test.append(total_data_objects[i].label)

X_test=np.array(X_test)
X_train=np.asarray(X_train)
#print("X_test",X_test)
#print("x_trin shape",X_train.shape)
#print("X_train",X_train[0].shape)    

Y_test=np.asarray(Y_test)
Y_train=np.asarray(Y_train)
Y_test=Y_test
Y_train=Y_train
Y_test=Y_test.reshape(-1,1)
Y_train=Y_train.reshape(-1,1)
#print(len(Y_train))
#print(Y_train[1:20,:])

In [ ]:
print("shape ",X_train.shape)
print("shape ",Y_train.shape)
gnb = GaussianNB()
gnb.fit(X_train,Y_train)
pred_labels=gnb.predict(X_test)
print("labels predicted",len(pred_labels))
print(Y_test.reshape(1,-1))
print(pred_labels)

In [ ]:
#performance
print("accuracy =",accuracy_score(pred_labels,Y_test)*100)

In [ ]:
cm = confusion_matrix(pred_labels,Y_test)
print(cm)

In [ ]:
auc = roc_auc_score(Y_test,pred_labels)
print('AUC: %.2f' % auc)
fpr, tpr, thresholds = roc_curve(Y_test,pred_labels)
plot_roc_curve(fpr, tpr)